In [25]:
import pandas as pd
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, HalvingRandomSearchCV, GridSearchCV, RandomizedSearchCV
from sklearn.decomposition import PCA, KernelPCA, FastICA
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from enum import Enum

from sklearn.linear_model import HuberRegressor

# Define an Enum class
class ML_TASK(Enum):
    CLASSIFICATION = 1
    REGRESSION = 2
    CLUSTERING = 3

class HYPERPARAM_SEARCH(Enum):
    RANDOM = 1
    GRID = 2
    HALVING_RANDOM = 3
    HALVING_GRID = 4

class DIM_REDUCTION_METHOD(Enum):
    PCA = 1
    KERNEL_PCA = 2
    ICA = 3

# Params
algo_name = 'RobustHuberRegression'
ml_task = ML_TASK.REGRESSION # True if you want to map fractions to integers for classification
hyperparam_search_method = HYPERPARAM_SEARCH.GRID # True if you want to use Random Grid Search instead of brute force
dim_reduction_method = DIM_REDUCTION_METHOD.ICA # False if you don't want to use PCA
kfold_num = 10 # Train-Test-Split = 1, LOOCV = n

# Read
X_train = pd.read_csv('../pc_X_train.csv')
y_train = pd.read_csv('../pc_y_train.csv')
y_train = y_train.iloc[:, -1] # With iloc we extract the labels

X_test = pd.read_csv('../pc_X_test.csv')
ids = X_test.iloc[:, 0]

# If classification -> Do mapping
if ml_task == ML_TASK.CLASSIFICATION:
    possible_numbers = list(set(y_train))
    mapping = {val: idx for idx, val in enumerate(possible_numbers)}
    y_train = [mapping[val] for val in y_train]


In [26]:
# Normalize Trainset
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_train)

# PCA
dim_reduction = None
match dim_reduction_method:
    case DIM_REDUCTION_METHOD.PCA:
        dim_reduction = PCA()
    case DIM_REDUCTION_METHOD.KERNEL_PCA:
        dim_reduction = KernelPCA()
    case DIM_REDUCTION_METHOD.ICA:
        dim_reduction = FastICA()

# If classification
model = None
scoring_method = None
if ml_task == ML_TASK.CLASSIFICATION:
    scoring_method = 'f1_weighted'
    raise ValueError("Linear Regression is a regression algorithm. Please use ML_TASK.REGRESSION")
elif ml_task == ML_TASK.REGRESSION:
    model = HuberRegressor()
    scoring_method = 'neg_root_mean_squared_error'
    # raise ValueError("x is a classification algorithm. Please use ML_TASK.x")

# Pipeline
steps = []
steps.append(('dim_reduction', dim_reduction))
steps.append(('model', model))

pipe = Pipeline(steps=steps)

# parameters = {'dim_reduction__n_components': [1,2,4,8,16], 
#               'model__criterion': ['gini', 'entropy', 'log_loss'], 
#               'model__splitter': ['best', 'random'],
#               'model__min_samples_split': [1,2,4,8,16,32],
#               'model__max_depth': [1,2,3,4,5,6,7,8,16,32,64]}

parameters = {'dim_reduction__n_components': [1,2,4,8,16,32],
              'model__fit_intercept': [True, False],
              'model__epsilon': [1.0, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 3.0],
              'model__alpha': [0.001, 0.0001, 0.00001]}

# Grid Search
grid_params_list = [pipe, parameters] 
grid_params_dict = {'cv': kfold_num, 'scoring': scoring_method, 'verbose':2, 'n_jobs':6}

grid = None
match hyperparam_search_method:
    case HYPERPARAM_SEARCH.GRID:
        grid = GridSearchCV(*grid_params_list, **grid_params_dict)
    case HYPERPARAM_SEARCH.RANDOM:
        grid = RandomizedSearchCV(*grid_params_list, **grid_params_dict)
    case HYPERPARAM_SEARCH.HALVING_GRID:
        grid = HalvingGridSearchCV(*grid_params_list, **grid_params_dict)
    case HYPERPARAM_SEARCH.HALVING_RANDOM:
        grid = HalvingRandomSearchCV(*grid_params_list, **grid_params_dict)

grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)
print("Best Score:", grid.best_score_)

model = grid.best_estimator_

# Normalize Testset
X_normalized_test = scaler.fit_transform(X_test)
X_reduced_test = dim_reduction.fit_transform(X_normalized_test)

# Predict
predictions = model.predict(X_reduced_test)

# If classification -> Transform back to Regression Task
if ml_task == ML_TASK.CLASSIFICATION:
    predictions = [possible_numbers[val] for val in predictions]

# Save 
output_filename = f'bork_GYCAOB_{algo_name}_C.csv' if ml_task == ML_TASK.CLASSIFICATION else f'bork_GYCAOB_{algo_name}_R.csv'
results_df = pd.DataFrame({'id': ids,'score': predictions})
results_df.to_csv(output_filename, index=False)

Fitting 10 folds for each of 396 candidates, totalling 3960 fits
[CV] END dim_reduction__n_components=1, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=True; total time=   0.1s
[CV] END dim_reduction__n_components=1, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=True; total time=   0.1s
[CV] END dim_reduction__n_components=1, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=True; total time=   0.1s
[CV] END dim_reduction__n_components=1, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=True; total time=   0.1s
[CV] END dim_reduction__n_components=1, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=1, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=1, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=1, model__alpha=

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterati

[CV] END dim_reduction__n_components=16, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=16, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=16, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=False; total time=   0.2s
[CV] END dim_reduction__n_components=16, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=16, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=False; total time=   0.2s
[CV] END dim_reduction__n_components=16, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=16, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=16, model__alpha=0.001, model__epsilon=1.0, model__fit_intercept=False; 

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END dim_reduction__n_components=16, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=False; total time=   0.2s
[CV] END dim_reduction__n_components=16, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=16, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=16, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=False; total time=   0.2s
[CV] END dim_reduction__n_components=16, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=False; total time=   0.2s
[CV] END dim_reduction__n_components=16, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=False; total time=   0.2s
[CV] END dim_reduction__n_components=16, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=False; total time=   0.1s
[CV] END dim_reduction__n_components=16, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=Fals

...

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterati

[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=False; total time=   0.2s
[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=False; total time=   0.2s


/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterati

[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=False; total time=   0.2s
[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=False; total time=   0.2s
[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=False; total time=   0.2s
[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=True; total time=   0.2s
[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=False; total time=   0.2s
[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=False; total time=   0.2s
[CV] END dim_reduction__n_components=32, model__alpha=1e-05, model__epsilon=1.0, model__fit_intercept=Fals

/opt/homebrew/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but FastICA was fitted with feature names
  warnings.warn(
